In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

gpus = tf.config.list_physical_devices('GPU')
if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], enable=True)

In [2]:
print("Num GPUs available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs available:  1


In [3]:
from datasets import load_dataset

cnn_daily_dataset = load_dataset('cnn_dailymail', '3.0.0')
print(cnn_daily_dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [4]:
train_df = pd.DataFrame.from_dict(cnn_daily_dataset['train'])
val_df = pd.DataFrame.from_dict(cnn_daily_dataset['validation'])
test_df = pd.DataFrame.from_dict(cnn_daily_dataset['test'])

In [5]:
df = pd.concat([train_df, test_df, val_df]).drop(columns='id')
df['id'] = df.index
df = df.iloc[:, [2, 0, 1]]

In [6]:
# print(df.head()['article'])
df

,id,article,highlights
0,0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."
...,...,...,...
13363,13363,"It is the dream of many young children, the ch...","The town in Valley Center, San Diego, has been..."
13364,13364,It’s the type of encounter that can send panic...,Photographer Graham Hewer captured the jaw-dro...
13365,13365,A group of tourists to the Bahamas enjoyed one...,The pigs swim through the crystal clear sea an...
13366,13366,Pippa Middleton bundled up against the London ...,Kate's sister is back in London following 'cri...


In [7]:
df['sent_count'] = df['article'].apply(len)
print(f'Min number of sentences: {df["sent_count"].min()}, Max num of sent.: {df["sent_count"].max()}')
df = df.drop(columns='sent_count')

Min number of sentences: 48, Max num of sent.: 15925


## Extractive phase

In [8]:
import networkx as nx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
nlp = spacy.load('en_core_web_sm')


def extract_sentences(text):
    doc = nlp(text)
    sents = [sent.text.strip() for sent in doc.sents]

    return sents


def get_similarity_matrix(sentences):
    tf_idf = TfidfVectorizer().fit_transform(sentences)
    similarity_matrix = cosine_similarity(tf_idf)

    return similarity_matrix


def get_graph(sim_matrix):
    nsents = len(sim_matrix)
    graph = nx.Graph()

    for i in range(nsents):
        for j in range(i + 1, nsents):
            if sim_matrix[i, j] > 0:
                graph.add_edge(i, j, weight=sim_matrix[i, j])

    return graph


def get_n_best_sents(text, nsents: int):
    sentences = extract_sentences(text)
    similarity_matrix = get_similarity_matrix(sentences)
    graph = get_graph(similarity_matrix)
    scores = nx.pagerank(graph, weight='weight')

    ranked_sentences = sorted(
        [(scores[i] if i in scores else 0, s) for i, s in enumerate(sentences)],
        reverse=True
    )
    top_sentences = [s for _, s in ranked_sentences[:nsents]]

    return top_sentences


if __name__ == '__main__':
    # print(str(df.head()['article'].values))
    top_sents = get_n_best_sents(str(df.head()['article'].values), 3)
    print(top_sents)

['In an additional summary of facts, signed by Vick and filed with the agreement, Vick admitted buying pit bulls and the property used for training and fighting the dogs, but the statement said he did not bet on the fights or receive any of the money won.', 'His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films.', 'They were in the middle of the Mississippi River, which was churning fast, and he had no way of getting to them.']


In [10]:
from parallel_pandas import ParallelPandas

SENT_COUNT_THRESHOLD = 25
TRIMMED_FILEPATH = 'datasets/trimmed_th_cnn_dailymail.h5'


def process_row(sent_list):
    half = len(sent_list) // 2
    output_sent_count = half if half < SENT_COUNT_THRESHOLD else SENT_COUNT_THRESHOLD
    best_sents = get_n_best_sents(sent_list, output_sent_count)

    return best_sents


def trimm_dataset(df):    
    ParallelPandas.initialize(n_cpu=os.cpu_count() - 2, split_factor=60, disable_pr_bar=False)
    output_df = df['article'].p_apply(lambda text: process_row(text))
    
    return output_df


trimmed = None

if __name__ == '__main__':
    if os.path.exists(TRIMMED_FILEPATH):
        print('Preprocessed dataset exists. Loading...')
        trimmed = pd.read_hdf(TRIMMED_FILEPATH, key='dataset')
        print('Data loaded')
    else:
        print('Processing a dataset...')
        trimmed = trimm_dataset(df)
        trimmed.to_hdf(TRIMMED_FILEPATH, key='dataset', mode='w')

Preprocessed dataset exists. Loading...
Data loaded


In [12]:
# trimmed = pd.read_hdf('datasets/trimmed_th_cnn_dailymail.h5', key='dataset')

---

## Abstractive phase

In [67]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Bidirectional, Layer
from tensorflow.keras.models import Model

# VOCAB_SZ = 5000
# EMBEDDING_DIM = 256
# UNITS = 256
# MAX_SEQ_LEN = 50

VOCAB_SZ = 5000
EMBEDDING_DIM = 256
UNITS = 256
MAX_SEQ_LEN = 50


def create_encoder(vocab_size, embedding_dim, units):
    encoder_inputs = Input(shape=(MAX_SEQ_LEN,), name="encoder_inputs")
    embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name="encoder_embedding")(encoder_inputs)
    
    encoder_rnn = Bidirectional(
        GRU(units, return_sequences=True, return_state=True, name="encoder_rnn"),
        name='bidirectional_encoder'
    )
    encoder_outputs, forward_state, backward_state = encoder_rnn(embedding)
    last_hidden_state = tf.concat([forward_state, backward_state], axis=-1)
    
    return encoder_inputs, encoder_outputs, last_hidden_state


class AttentionLayer(Layer):
    def __init__(self, units):
        super(AttentionLayer, self).__init__()
        self.W_alpha = Dense(units)
        self.U_alpha = Dense(units)
        self.Z_alpha = Dense(1)

    def call(self, decoder_state, encoder_outputs):
        decoder_state_expanded = tf.expand_dims(decoder_state, 1)
        alignment_scores = self.Z_alpha(
            tf.nn.tanh(self.W_alpha(decoder_state_expanded) + self.U_alpha(encoder_outputs))
        )
        attention_weights = tf.nn.softmax(alignment_scores, axis=1)
        context_vector = tf.reduce_sum(attention_weights * encoder_outputs, axis=1)
        
        return context_vector, attention_weights


class PointerDecoder(Layer):
    def __init__(self, vocab_size, embedding_dim, units):
        super(PointerDecoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, name="decoder_embedding")
        self.gru = GRU(units * 2, return_sequences=True, return_state=True, name="decoder_rnn")
        self.attention = AttentionLayer(units * 2)
        self.vocab_dense = Dense(vocab_size, activation="softmax", name="vocab_output")
        self.pointer_switch = Dense(1, activation="sigmoid", name="pointer_switch")

    def call(self, decoder_inputs, encoder_outputs, initial_state, encoder_inputs):
        all_outputs = []
        state = initial_state
        embedding = self.embedding(decoder_inputs)

        for t in range(MAX_SEQ_LEN):
            context_vector, attention_weights = self.attention(state, encoder_outputs)

            expanded_state = tf.expand_dims(state, axis=1)
            pointer_logits = self.pointer_switch(tf.concat([expanded_state, tf.expand_dims(context_vector, 1)], axis=-1))
            pointer_probs = tf.squeeze(pointer_logits, axis=-1)

            decoder_input_t = tf.concat([embedding[:, t:t + 1, :], tf.expand_dims(context_vector, 1)], axis=-1)
            output, state = self.gru(decoder_input_t, initial_state=state)

            vocab_probs = self.vocab_dense(output)

            encoder_inputs_int = tf.cast(encoder_inputs, tf.int32)
            pointer_output = tf.reduce_sum(attention_weights * tf.one_hot(encoder_inputs_int, depth=self.vocab_dense.units), axis=1)

            final_output = pointer_probs * pointer_output + (1 - pointer_probs) * tf.squeeze(vocab_probs, axis=1)
            all_outputs.append(tf.expand_dims(final_output, axis=1))

        decoder_outputs = tf.concat(all_outputs, axis=1)
        
        return decoder_outputs


def create_model(vocab_size, embedding_dim, units):
    encoder_inputs, encoder_outputs, encoder_last_state = create_encoder(vocab_size, embedding_dim, units)

    decoder_inputs = Input(shape=(MAX_SEQ_LEN,), name="decoder_inputs")
    pointer_decoder = PointerDecoder(vocab_size, embedding_dim, units)
    decoder_outputs = pointer_decoder(
        decoder_inputs, encoder_outputs, encoder_last_state, encoder_inputs
    )

    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs, name='bidirectional_encoder_decoder')
    
    return model

In [68]:
model = create_model(VOCAB_SZ, EMBEDDING_DIM, UNITS)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "bidirectional_encoder_decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 50)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 50, 256)      1280000     ['encoder_inputs[0][0]']         
                                                                                                  
 bidirectional_encoder (Bidirec  [(None, 50, 512),   789504      ['encoder_embedding[0][0]']      
 tional)                         (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                      

In [54]:
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

START_TOKEN = '<start>'
END_TOKEN = '<end>'

DATASET_SZ = len(trimmed)
TRAIN_TEST_RATIO = 0.8
TRAIN_SZ = int(TRAIN_TEST_RATIO * DATASET_SZ)

train_source_articles = trimmed[:TRAIN_SZ]
train_target_articles = df['highlights'][:TRAIN_SZ]

print(train_source_articles.shape, train_target_articles.shape)

# 3 examples for testing
test_source_articles = trimmed[TRAIN_SZ : TRAIN_SZ + 3]
test_target_articles = df['highlights'][TRAIN_SZ : TRAIN_SZ + 3]

print(test_source_articles.shape, test_target_articles.shape)

(20000,) (20000,)
(3,) (3,)


In [55]:
tokenizer = Tokenizer(num_words=VOCAB_SZ, oov_token='<OOV>')
tokenizer.fit_on_texts(pd.concat([train_source_articles, train_target_articles], axis=0))
tokenizer.word_index.update({START_TOKEN: 1, END_TOKEN: 2})

# Preparing training data
encoder_input_data = tokenizer.texts_to_sequences(train_source_articles)
decoder_input_data = tokenizer.texts_to_sequences(START_TOKEN + ' ' + train_target_articles)
decoder_target_data = tokenizer.texts_to_sequences(train_target_articles + ' ' + END_TOKEN)

encoder_input_data = pad_sequences(encoder_input_data, maxlen=MAX_SEQ_LEN, padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=MAX_SEQ_LEN, padding='post')
decoder_target_data = pad_sequences(decoder_target_data, maxlen=MAX_SEQ_LEN, padding='post')
decoder_target_data = np.expand_dims(decoder_target_data, -1)

# Preparing test data
test_encoder_input_data = tokenizer.texts_to_sequences(test_source_articles)
test_decoder_target_data = tokenizer.texts_to_sequences(test_target_articles + ' ' + END_TOKEN)

test_encoder_input_data = pad_sequences(test_encoder_input_data, maxlen=MAX_SEQ_LEN, padding='post')
test_decoder_target_data = pad_sequences(test_decoder_target_data, maxlen=MAX_SEQ_LEN, padding='post')

print(f'encoder_input_data.shape={encoder_input_data.shape}, decoder_input_data.shape={decoder_input_data.shape}, decoder_target_data.shape={decoder_target_data.shape}\n')
print(f'Test data shapes -> encoder_input={test_encoder_input_data.shape}, decoder_target={test_decoder_target_data.shape}')

encoder_input_data.shape=(20000, 50), decoder_input_data.shape=(20000, 50), decoder_target_data.shape=(20000, 50, 1)

Test data shapes -> encoder_input=(3, 50), decoder_target=(3, 50)


In [59]:
len(tokenizer.word_index)

125698

In [66]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join('model', 'max_vocab_sz_e{epoch:02d}_val_loss_{val_loss:.2f}.h5'),
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
    save_best_only=False,
    verbose=1
)

history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=8,
    epochs=30,
    validation_split=0.2,
    callbacks=[checkpoint_callback]
)

Epoch 1/30


2025-01-20 17:27:35.543146: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 208000000 exceeds 10% of free system memory.
2025-01-20 17:27:36.487228: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 208000000 exceeds 10% of free system memory.
2025-01-20 17:27:53.060051: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 49.59MiB (rounded to 52000000)requested by op gradient_tape/sparse_categorical_crossentropy/clip_by_value/LessEqual
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-01-20 17:27:53.061681: W tensorflow/core/common_runtime/bfc_allocator.cc:474] **************************************x**********************x****************************x*********
2025-01-20 17:27:53.063383: W tensorflow/core/framework/op_kernel.cc:1

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sparse_categorical_crossentropy/clip_by_value/LessEqual' defined at (most recent call last):
    File "/home/krzys/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/krzys/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/krzys/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/krzys/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/krzys/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/krzys/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 205, in start
      

In [20]:
model.load_weights('model/model_epoch_20_val_loss_5.65.h5')
history_extended = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=16,
    epochs=30,
    initial_epoch=20,
    validation_split=0.2,
    callbacks=[checkpoint_callback]
)

Epoch 21/30
1000/1000 [==============================] - ETA: 0s - loss: 0.9812 - accuracy: 0.7478
Epoch 21: saving model to model/model_epoch_21_val_loss_5.73.h5
1000/1000 [==============================] - 280s 279ms/step - loss: 0.9812 - accuracy: 0.7478 - val_loss: 5.7268 - val_accuracy: 0.3140
Epoch 22/30
1000/1000 [==============================] - ETA: 0s - loss: 0.9323 - accuracy: 0.7585
Epoch 22: saving model to model/model_epoch_22_val_loss_5.82.h5
1000/1000 [==============================] - 279s 279ms/step - loss: 0.9323 - accuracy: 0.7585 - val_loss: 5.8180 - val_accuracy: 0.3134
Epoch 23/30
1000/1000 [==============================] - ETA: 0s - loss: 0.8909 - accuracy: 0.7667
Epoch 23: saving model to model/model_epoch_23_val_loss_5.91.h5
1000/1000 [==============================] - 283s 283ms/step - loss: 0.8909 - accuracy: 0.7667 - val_loss: 5.9112 - val_accuracy: 0.3100
Epoch 24/30
1000/1000 [==============================] - ETA: 0s - loss: 0.8580 - accuracy: 0.7724
E

In [69]:
from rouge import Rouge
from tqdm import tqdm


def summarize_text(model, tokenizer, input_text, max_len=MAX_SEQ_LEN):    
    start_token = tokenizer.word_index[START_TOKEN]
    end_token = tokenizer.word_index[END_TOKEN]
    decoder_input = np.zeros((1, max_len))
    decoder_input[0, 0] = start_token
    
    if len(input_text.shape) == 1:
        input_text = np.expand_dims(input_text, axis=0)
    
    summary = []
    for i in range(max_len):
        decoder_output = model.predict([input_text, decoder_input])
        predicted_id = tf.argmax(decoder_output[:, -1, :], axis=-1).numpy()[0]
        
        if predicted_id == end_token:
            break
            
        summary.append(predicted_id)
        decoder_input[0, i] = predicted_id
    
    decoded_summary = tokenizer.sequences_to_texts([summary])[0]
    print(decoded_summary)
        
    return decoded_summary


def evaluate_model(model, tokenizer, test_inputs, test_targets, max_len=MAX_SEQ_LEN):
    predictions = []
    for article in tqdm(test_inputs, desc='Evaluating model on test set'):
        summary = summarize_text(model, tokenizer, article, max_len)
        predictions.append(summary)

    rouge = Rouge()
    rouge_scores = rouge.get_scores(predictions, test_targets, avg=True)
    
    return rouge_scores, predictions


weights_path = 'model/model_epoch_30_val_loss_6.38.h5'
model.load_weights(weights_path)

rouge_scores, predictions = evaluate_model(model, tokenizer, test_encoder_input_data, test_decoder_target_data)

print(f'Average ROUGE scores: {rouge_scores}')

Evaluating model on test set:  33%|███▎      | 1/3 [00:36<01:13, 36.60s/it]

<OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>


Evaluating model on test set:  67%|██████▋   | 2/3 [00:48<00:21, 21.86s/it]

<OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>


Evaluating model on test set: 100%|██████████| 3/3 [01:00<00:00, 20.02s/it]

<OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>


AssertionError: 